# Edge Detection Techniques.

In [24]:
import cv2
import numpy as np
from tkinter import *
from PIL import Image, ImageTk

# Open video capture object
cap = cv2.VideoCapture(0)

# Check if video capture is open
if not cap.isOpened():
    print('Error: Unable to open video capture')
    exit()

# Create Tkinter window
root = Tk()

# Create label to display frame
label = Label(root)
label.pack()

# Selected filter
selected_filter = 'Original'

# Button callback function
def on_button(filter):
    global selected_filter
    selected_filter = filter

# Create buttons for each filter
Button(root, text='Original', command=lambda: on_button('Original')).pack(side=LEFT)
Button(root, text='Canny', command=lambda: on_button('Canny')).pack(side=LEFT)
Button(root, text='Sobel', command=lambda: on_button('Sobel')).pack(side=LEFT)
Button(root, text='Prewitt', command=lambda: on_button('Prewitt')).pack(side=LEFT)
Button(root, text='Robinson Mask', command=lambda: on_button('Robinson Mask')).pack(side=LEFT)
Button(root, text='Laplacian', command=lambda: on_button('Laplacian')).pack(side=LEFT)

#Prewitt Kernels

#kernel for x axis
kernelx = np.array([[1,1,1],[0,0,0],[-1,-1,-1]])
#kernel for y axis
kernely = np.array([[-1,0,1],[-1,0,1],[-1,0,1]])

#Roninson Mask Kernels
kernel_rob_north = np.array([[-1,0,1],[-2,0,2],[-1,0,1]])
kernel_rob_west = np.array([[1,2,1],[0,0,0],[1,2,1]])
kernel_rob_east = np.array([[-1,-2,-1],[0,0,0],[1,2,1]])
kernel_rob_south = np.array([[1,0,-1],[2,0,-2],[1,0,-1]])


# Update function
def update():
    # Read frame from camera
    ret, frame = cap.read()

    # Check if frame was read successfully
    if not ret:
        print('Error: Unable to read frame from camera')
        return

    # Apply selected filter to frame
    if selected_filter == 'Canny':
        gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
        frame = cv2.Canny(gray, 35, 75)

    elif selected_filter == 'Sobel':
        gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
        img_sobelx = cv2.Sobel(gray,cv2.CV_8U,1,0,ksize=3)
        img_sobely = cv2.Sobel(gray,cv2.CV_8U,0,1,ksize=3)
        frame = img_sobelx + img_sobely
        
       

    elif selected_filter == 'Prewitt':
        gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
        img_prewittx = cv2.filter2D(gray, -1, kernelx)
        img_prewitty = cv2.filter2D(gray, -1, kernely)
        frame = img_prewittx + img_prewitty

       

    elif selected_filter == 'Robinson Mask':
        gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
        img_rob_north = cv2.filter2D(gray,-1,kernel_rob_north)
        img_rob_south = cv2.filter2D(gray,-1,kernel_rob_south)
        img_rob_east = cv2.filter2D(gray,-1,kernel_rob_east)
        img_rob_west = cv2.filter2D(gray,-1,kernel_rob_west)

        frame = img_rob_east + img_rob_north + img_rob_west + img_rob_south
        # frame = cv2.bitwise_not(frame)

    elif selected_filter == 'Laplacian':
        gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
        frame = cv2.Laplacian(gray, cv2.CV_64F)
        frame = cv2.convertScaleAbs(frame)

    # Convert frame to PIL image
    try:
        frame_pil = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    except Exception as e:
        print('Error: Unable to convert frame to PIL image:', e)
        return

    # Convert PIL image to PhotoImage
    try:
        frame_photo = ImageTk.PhotoImage(image=frame_pil)
    except Exception as e:
        print('Error: Unable to convert PIL image to PhotoImage:', e)
        return

    # Update label with PhotoImage
    label.config(image=frame_photo)

    # Keep reference to PhotoImage
    label.image = frame_photo

    # Schedule next update
    root.after(10, update)

# Schedule first update
root.after(10, update)

# Start Tkinter main loop
root.mainloop()

# Release video capture object
cap.release()
